In [20]:
import pandas as pd
import itertools
import numpy as np
import openpyxl
# from scenario_generation import generate_fuel_prices


### SETS

In [21]:
fuel_price_csv = 'fuel_price2_27s.csv'
# Velg antall av de ulike settene her
#set_sheets = {'Power systems': 11, 'Ages': 6, 'Routes': 3122, 'Time periods':6, 'Time periods 1': 2, 'Time periods 2':4, 'Fuel types': 10, 'Installations': 18, 'Scenarios': 100}

t_values = range(0, 6)  # time period from 1 to 6
t1_values = range(0, 2)
t2_values = range(2, 6)
tp2_values = range(2,7)
f_values = range(1, 10)  # Assuming you want values from 1 to 7 for 'f'
r_values = range(1, 58)  # Assuming you want values from 1 to 3122 for 'r'
n_values = range(1, 19) # 18 installations
s_values = range(1,12) # 11 systems
w_values = range(1, 11) # 100 scenarios
a_values = range(0, 6)

for t in f_values:
    print(t)


1
2
3
4
5
6
7
8
9


In [22]:
# Skriver alle sets med verdier definert over til excel

sets_excel_output_path = '../run_model/SetData-sheets.xlsx'

set_sheets = {
    'Power systems': s_values,
    'Ages': a_values,
    'Routes': r_values,
    'Time periods': t_values,
    'Time periods 1': t1_values,
    'Time periods 2': t2_values,
    'TP2 scrap': tp2_values,
    'Fuel types': f_values,
    'Installations': n_values,
    'Scenarios': w_values
}

# Create data frames for each set and write to Excel
with pd.ExcelWriter(sets_excel_output_path, engine='openpyxl') as writer:
    for sheet, values in set_sheets.items():
        # Create a DataFrame for the current set
        df = pd.DataFrame(itertools.product(values), columns=[sheet])

        # Write the DataFrame to the Excel file with the current sheet name
        df.to_excel(writer, sheet_name=sheet, index=False)

#### Generere kombinasjoner av indekser til parameterne

In [23]:

combinations_f_r = itertools.product(f_values, r_values)
combinations_ems = itertools.product(f_values, r_values)
combinations_r_s = itertools.product(r_values, s_values)
combinations_t = itertools.product(t_values)
combinations_s_a = itertools.product(s_values, a_values)
combinations_f_r_w = itertools.product(f_values, r_values, w_values)
combinations_w = itertools.product(w_values)
combinations_r = itertools.product(r_values)

#### Inputdata

In [24]:
# Hardkodet inn fuelpris per MJ per fueltype 1-6
fuel_prices = {1: 0.171, 2:0.163, 3:0.514, 4:0.328, 5:0.222, 6:0.205, 7: 0.197, 8: 0.183, 9: 0.297}




# fuel_dict = {
#     1: (0.178, 0.094, 'fossil'),      # MGO
#     2: (0.146, 0.073, 'fossil'),      # LNG
#     3: (0.491, 0.261, 'green_fuels'), # GREEN METHANOL
#     4: (0.324, 0.178, 'green_fuels'), # GREEN NH3
#     5: (0.278, 0.147, 'bio'),         # B30
#     6: (0.245, 0.130, 'bio'),         # B20
#     7: (0.228, 0.121, 'bio'),         # B15
#     8: (0.201, 0.106, 'bio'),         # B7
#     9: (0.301, 0.129, 'green_fuels')  # Helelektrisk
# }
 # Example fuel bounds with group information


# Karbonutslipp per MJ som er brukt for hver type fuel
emissions_carbon_tonn_per_MJ_wtw = {1: 0.000182, 2: 0.000153, 3: 0.000033, 4: 0.000033, 5: 0.000134, 6: 0.000150, 7: 0.000158, 8: 0.000170, 9: 0.000008 }
emissions_carbon_tonn_per_MJ_ttw = {1: 0.000152778, 2: 0.000115556, 3: 0.000085278, 4: 0.000025583, 5: 0.000017528, 6: 0.0, 7: 0.0}
emissions_carbon_tonn_per_MJ = emissions_carbon_tonn_per_MJ_wtw 


# Regne ut reduksjonskrav for karbonutslipp
init_ems_tonn = 165000

scenario_1_reductions = {0: 0, 1: 0.50, 2: 0.55, 3: 0.70, 4: 0.80, 5: 0.80 } # ambisjonene til norske myndigheter. t=1 = 2030
scenario_2_reductions = {0: 0, 1: 0, 2: 0.29, 3: 0.49, 4: 0.65, 5: 0.78, 6: 0.87} # DNV report Norway 2023 - kutt kun innenfor transport
scenario_0_reductions = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

# Velg hvilket scenario som skal kjøres
reduction_requirements = scenario_1_reductions

capacity_systems = {1: 100, 2: 100, 3: 100, 4: 100, 5: 70, 6: 70, 7: 70, 8: 70, 9: 90, 10: 90, 11: 70, 12: 90, 13: 90}

#### Lese inn data fra rutegenereringsprogrammet

In [25]:
# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv('../route_generation_mongstad/generated_datafiles_reduced/mj_route.csv')
#mj_route_el = pd.read_csv('../route_generation/generated_datafiles_reduced/mj_route_strøm.csv')



# # Leser inn tid på å kjøre hver rute
hours_route = pd.read_csv('../route_generation_mongstad/generated_datafiles_reduced/hours_route.csv')
#hours_route_el = pd.read_csv('../route_generation/generated_datafiles_reduced/hours_route_strøm.csv')

#demand_el = pd.read_csv('../route_generation/generated_datafiles_reduced/demand_strøm.csv')
demand = pd.read_csv('../route_generation_mongstad/generated_datafiles_reduced/demand.csv')

# Skriver verdiene tilbake i excelfilen for alle parametrene
output_excel_path = '../run_model/Parameterdata-sheets.xlsx'


##### Generere P_w

In [26]:
df_probability = pd.DataFrame(combinations_w, columns=['O'])

df_probability['Value'] = 1/len(list(w_values))

df_probability.head()


,O,Value
0,1,0.1
1,2,0.1
2,3,0.1
3,4,0.1
4,5,0.1


In [27]:

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_probability.to_excel(writer, sheet_name='Probability', index=False)

In [28]:
distance_dict = {}

for idx, value in enumerate(mj_route,1):
    distance_dict[idx] = value

print(distance_dict)

{1: 'mj_route'}


##### Distance dict for compatibility og cost of fuel

In [29]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
print(distance_dict)
# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in enumerate(distance_dict.values())}
print(len(distance_dict))
print(distance_dict)


{0: 221758.4, 1: 178953.6, 2: 192976.8, 3: 220346.8, 4: 210389.2, 5: 200084.8, 6: 265296.4, 7: 244674.0, 8: 240312.0, 9: 201093.2, 10: 205730.8, 11: 194600.4, 12: 193478.0, 13: 204507.6, 14: 280065.2, 15: 226675.6, 16: 240136.4, 17: 215881.2, 18: 241484.4, 19: 231120.4, 20: 233664.8, 21: 279163.6, 22: 212046.4, 23: 207487.2, 24: 227290.8, 25: 202117.2, 26: 182802.0, 27: 249453.2, 28: 261579.6, 29: 254460.8, 30: 225621.2, 31: 242612.0, 32: 229348.0, 33: 266538.8, 34: 253531.6, 35: 241464.4, 36: 266200.0, 37: 262597.2, 38: 243741.2, 39: 183554.4, 40: 206008.0, 41: 259612.4, 42: 257894.4, 43: 212159.2, 44: 206875.2, 45: 206267.6, 46: 191340.0, 47: 222956.4, 48: 227516.4, 49: 212476.4, 50: 213384.0, 51: 186510.0, 52: 205663.6, 53: 187422.4, 54: 197423.6, 55: 208425.2, 56: 230912.4}
57
{1: 221758.4, 2: 178953.6, 3: 192976.8, 4: 220346.8, 5: 210389.2, 6: 200084.8, 7: 265296.4, 8: 244674.0, 9: 240312.0, 10: 201093.2, 11: 205730.8, 12: 194600.4, 13: 193478.0, 14: 204507.6, 15: 280065.2, 16: 22

##### Compatibility RS

In [30]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
demand_dict = demand['Demand'].to_dict()
#demand_dict = demand_dict['Demand']

# lager egne distanser for elektrisk

#specific_demand_dict = demand_el['Demand'].to_dict()
#specific_demand_dict= specific_demand_dict['Demand']

#electric_demand_dict = {key + 1: value for key, value in specific_demand_dict.items()}

demand_dict = {key + 1: value for key, value in demand_dict.items()}




In [31]:
# Lage combinations for rs
# For hver rad med r s, sjekke om kapasiteten i s er større enn eller lik demanden til r. Sett til 1 hvis true, 0 ellers

# Create a DataFrame from the combinations
df_routecap = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Display the first few rows of the DataFrame
print(df_routecap.head())

df_routecap['Value'] = 0

# Iterate over the DataFrame
for index, row in df_routecap.iterrows():
    r = row['R']
    s = row['S']
    if demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
        df_routecap.at[index, 'Value'] = 1
    # if s == 11 and electric_demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
    #     df_routecap.at[index, 'Value'] = 1

#print(df_routecap)

   R  S
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [32]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_routecap.to_excel(writer, sheet_name='Compatibility rs', index=False)

#### Demand for set covering

In [33]:
df_demand = pd.DataFrame(list(demand_dict.items()), columns=['R', 'Value'])

print(df_demand)

     R   Value
0    1   85.25
1    2   76.00
2    3   69.00
3    4   65.00
4    5   96.25
5    6   75.42
6    7   75.50
7    8   68.00
8    9   72.00
9   10   79.00
10  11   65.00
11  12   75.50
12  13   63.00
13  14   98.00
14  15   89.75
15  16   93.67
16  17   75.50
17  18   61.00
18  19   93.50
19  20   88.50
20  21   92.00
21  22   86.50
22  23   79.50
23  24   63.50
24  25   95.25
25  26   96.25
26  27   68.00
27  28   79.00
28  29   86.50
29  30   87.00
30  31   79.00
31  32   78.50
32  33   78.50
33  34   89.50
34  35   86.50
35  36   88.50
36  37   95.00
37  38   79.00
38  39   99.17
39  40   93.17
40  41   95.00
41  42   82.00
42  43   91.00
43  44   88.00
44  45   98.75
45  46   98.00
46  47   97.17
47  48   99.25
48  49   99.25
49  50   99.25
50  51   96.50
51  52   99.00
52  53   98.67
53  54   98.17
54  55   98.67
55  56   97.00
56  57  100.00


In [34]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_demand.to_excel(writer, sheet_name='Demand route', index=False)

##### Cost of fuel per route

In [35]:

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
# #specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(len(distance_dict))

print(distance_dict)
#print(len(electric_distance_dict))

57
{1: 221758.4, 2: 178953.6, 3: 192976.8, 4: 220346.8, 5: 210389.2, 6: 200084.8, 7: 265296.4, 8: 244674.0, 9: 240312.0, 10: 201093.2, 11: 205730.8, 12: 194600.4, 13: 193478.0, 14: 204507.6, 15: 280065.2, 16: 226675.6, 17: 240136.4, 18: 215881.2, 19: 241484.4, 20: 231120.4, 21: 233664.8, 22: 279163.6, 23: 212046.4, 24: 207487.2, 25: 227290.8, 26: 202117.2, 27: 182802.0, 28: 249453.2, 29: 261579.6, 30: 254460.8, 31: 225621.2, 32: 242612.0, 33: 229348.0, 34: 266538.8, 35: 253531.6, 36: 241464.4, 37: 266200.0, 38: 262597.2, 39: 243741.2, 40: 183554.4, 41: 206008.0, 42: 259612.4, 43: 257894.4, 44: 212159.2, 45: 206875.2, 46: 206267.6, 47: 191340.0, 48: 222956.4, 49: 227516.4, 50: 212476.4, 51: 213384.0, 52: 186510.0, 53: 205663.6, 54: 187422.4, 55: 197423.6, 56: 208425.2, 57: 230912.4}


In [36]:
# Convert dictionaries to DataFrames
df_distance = pd.DataFrame(distance_dict.items(), columns=['R', 'Value'])
#df_specific_distance = pd.DataFrame(electric_distance_dict.items(), columns=['R', 'Value'])

# Sort DataFrames by 'R' values
df_distance = df_distance.sort_values(by='R')
#df_specific_distance = df_specific_distance.sort_values(by='R')


# print("Distance Dictionary DataFrame:")
# print(df_distance)

# print("\nSpecific Distance Dictionary DataFrame:")
# print(df_specific_distance)

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_distance.to_excel(writer, sheet_name='Distance', index=False)

# with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
#     df_specific_distance.to_excel(writer, sheet_name='Specific distance', index=False)

Generere alle kombinasjoner av R og F

In [37]:
# Create a DataFrame from the combinations
df_fuelprice = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_fuelprice.head())




   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


#### Fuelcost

In [38]:

# regner ut verdi for hver kombinasjon av fueltype og rute gitt verdiene i dictionariene over
# def calculate_value(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return fuel_prices[row['F']] * distance

def calculate_value(row):
    distance = distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_fuelprice['Value'] = df_fuelprice.apply(calculate_value, axis=1)

# df_fuelprice

##### Fuelcost 2

In [39]:
fuel_price2_df = pd.read_csv(fuel_price_csv)

fuel_price2_df.head()

,F,T,S,Value
0,1,2,1,0.174365
1,1,3,1,0.174365
2,1,4,1,0.174365
3,1,5,1,0.174365
4,1,2,2,0.154394


##### Skriver Cost of fuel til excel

In [40]:
# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    fuel_price2_df.to_excel(writer, sheet_name='Cost of fuel 2', index=False)

# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice.to_excel(writer, sheet_name='Cost of fuel 1', index=False)

##### EMISSIONS FOR FUEL TYPE

Generere alle kombinasjoner av F og R i kolonnene


In [41]:
# Create a DataFrame from the combinations
df_ems = pd.DataFrame(combinations_ems, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_ems.head())

   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [42]:
# multipliserer carbon per MJ per fueltype med distanse seilt per rute. 
# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance

# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance
def calculate_emissions(row):
    distance =  distance_dict[row['R']]
    return emissions_carbon_tonn_per_MJ[row['F']] * distance



# Apply the function to create a new column 'Value'
df_ems['Value'] = df_ems.apply(calculate_emissions, axis=1)


# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ems.to_excel(writer, sheet_name='Emissions', index=False)


##### MAX EMISSIONS PER TIME PERIOD

In [43]:


# Create a DataFrame from the combinations
df_max_ems = pd.DataFrame(combinations_t, columns=['T'])

# Display the first few rows of the DataFrame
# print(df_max_ems)

In [44]:


# Regner ut max emissions gitt uslippsreduksjonsscenario. Endre scenario-funksjon etter hvilket scenario vi vil kjøre med
def calculate_max_emissions(row):
    return (1 - reduction_requirements[row['T']]) * init_ems_tonn

# Legger til verdiene i Value-kolonne i dataframen
df_max_ems['Value'] = df_max_ems.apply(calculate_max_emissions, axis=1)
# print(df_max_ems.head())
# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_max_ems.to_excel(writer, sheet_name='Max Emissions', index=False)

df_max_ems

,T,Value
0,0,165000.0
1,1,82500.0
2,2,74250.0
3,3,49500.0
4,4,33000.0
5,5,33000.0


##### TIME USED

In [45]:
# Create T_rs
combinations_r_s = itertools.product(r_values, s_values)
df_timeused = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
route_dict = hours_route['Hours'].to_dict()
hours_route_dict = {key + 1: value for key, value in route_dict.items()}
print(hours_route_dict)
# lager egne distanser for elektrisk
# hours_route_el_dict = hours_route_el['Hours'].to_dict()
# hours_route_el_dict = {key + 1: value for key, value in hours_route_el_dict.items()}
# print(hours_route_el_dict)


def calculate_timeused(row):
    value =  hours_route_dict[row['R']]
    return value


df_timeused['Value'] = df_timeused.apply(calculate_timeused, axis=1)

print(df_timeused.head())
# Skriver dataframe med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_timeused.to_excel(writer, sheet_name='Time used', index=False)


{1: 48.0, 2: 48.0, 3: 48.0, 4: 48.0, 5: 48.0, 6: 48.0, 7: 48.0, 8: 48.0, 9: 48.0, 10: 48.0, 11: 48.0, 12: 48.0, 13: 48.0, 14: 48.0, 15: 48.0, 16: 48.0, 17: 48.0, 18: 48.0, 19: 48.0, 20: 48.0, 21: 48.0, 22: 48.0, 23: 48.0, 24: 48.0, 25: 48.0, 26: 48.0, 27: 48.0, 28: 48.0, 29: 48.0, 30: 48.0, 31: 48.0, 32: 48.0, 33: 48.0, 34: 48.0, 35: 48.0, 36: 48.0, 37: 48.0, 38: 48.0, 39: 48.0, 40: 48.0, 41: 48.0, 42: 48.0, 43: 48.0, 44: 48.0, 45: 48.0, 46: 48.0, 47: 48.0, 48: 48.0, 49: 48.0, 50: 48.0, 51: 48.0, 52: 48.0, 53: 48.0, 54: 48.0, 55: 48.0, 56: 48.0, 57: 48.0}
   R  S  Value
0  1  1   48.0
1  1  2   48.0
2  1  3   48.0
3  1  4   48.0
4  1  5   48.0


#### Installations in route

In [49]:
import csv

csv_file = "../route_generation_mongstad/generated_datafiles_reduced/routes.csv"
excel_file = '../run_model/Parameterdata-sheets.xlsx'

# Open the existing Excel workbook
workbook = openpyxl.load_workbook(excel_file)
sheet = workbook["Installations in route"]

# Open the CSV file in text mode and read all lines
with open(csv_file, 'r') as file:
    lines = file.readlines()

# Add column headers if necessary
if not sheet['A1'].value:
    sheet['A1'] = "i"
    sheet['B1'] = "r"
    sheet['C1'] = "Value"

# Initialize the row index for Excel
row_index = 2

# Initialize an empty dictionary to store the platform mapping
platform_mapping = {}

# Path to your CSV file
csv_file = "../route_generation_mongstad/clustering/output_platforms_demand.csv"

# Open the CSV file and read the contents
with open(csv_file, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        platform_mapping[platform] = len(platform_mapping) + 1

print(platform_mapping)


{'APT': 1, 'ASL': 2, 'DAB': 3, 'DSA': 4, 'DSS': 5, 'GFAGFBGFC': 6, 'KVB': 7, 'MID': 8, 'NLNVAL': 9, 'OSC': 10, 'OSE': 11, 'OSOVFB': 12, 'OSS': 13, 'STASTB': 14, 'STC': 15, 'TENTRB': 16, 'TEQTRC': 17, 'TRO': 18}


In [ ]:

platform_mapping_reverse = {v: k for k, v in platform_mapping.items()}

length = len(platform_mapping) +1

# Process each line in the CSV file
for idx, line in enumerate(lines, start=1):
    for i in range(1, length):
        sheet.cell(row=row_index, column=2, value=idx)
        sheet.cell(row=row_index, column=1, value=i)
        platform_name = platform_mapping_reverse.get(i, "")  # Get the platform name
        if platform_name in line:
            sheet.cell(row=row_index, column=3, value=1)
        else:
            sheet.cell(row=row_index, column=3, value=0)
        row_index += 1

workbook.save(excel_file)
print("Data has been successfully written.")

Data has been successfully written.


In [50]:

csv_file = '../route_generation_mongstad/clustering/output_platforms_visits.csv'


visits_per_week = {}
with open(csv_file, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    i = 1
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        # Assign a number to each platform
        visits_per_week[i] = row[1]
        i += 1

i_values = range(1, len(platform_mapping)+1)
print(visits_per_week)

combinations_i_t = itertools.product(i_values, t_values)
df_demand = pd.DataFrame(combinations_i_t, columns=['I', 'T'])
# Iterate over rows in the DataFrame
df_demand['Value'] = df_demand['I'].map(lambda x: visits_per_week[x])



df_demand

{1: '2', 2: '3', 3: '4', 4: '3', 5: '3', 6: '4', 7: '2', 8: '2', 9: '3', 10: '2', 11: '5', 12: '4', 13: '3', 14: '4', 15: '3', 16: '4', 17: '4', 18: '2'}


,I,T,Value
0,1,0,2
1,1,1,2
2,1,2,2
3,1,3,2
4,1,4,2
...,...,...,...
103,18,1,2
104,18,2,2
105,18,3,2
106,18,4,2


In [51]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_demand.to_excel(writer, sheet_name='Demand', index=False)